# The titanic data challenge

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load the data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
print(train_df[:3], '\n')
print(train_df.info())

## Test submissions
Test files:
* random prediction
* survival for women and children only

In [ ]:
# Create dummy random output for first submission
id_df = test_df['PassengerId']
random_df = pd.DataFrame(np.random.randint(low=0, high=2, size=(id_df.shape[0], 1)), columns=['Survived'])
result_df = pd.concat([id_df, random_df], axis=1)

# Save output to file
result_df.to_csv("result_random.csv", index=False, float_format='%.0f')

In [ ]:
# Create dummy output with survival for women and children only
result_df = test_df
result_df['Survived'] = ((result_df['Sex'] == 'female') | (result_df['Age'] < 16)).astype(int)

# Save output to file
result_df.to_csv("result_children_women.csv", columns=['PassengerId', 'Survived'], index=False, float_format='%.0f')